In [17]:
from datasets import load_dataset



# Load the dataset
dataset = load_dataset("conll2003", trust_remote_code=True)

train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']



In [18]:
print(train_dataset)

Dataset({
    features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})


In [19]:
labels = dataset["train"].features["ner_tags"].feature.names
num_labels = len(labels)
print(labels)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [20]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda


In [21]:
# from transformers import AutoModelForTokenClassification
# 
# model_checkpoint = "bert-base-cased"
# 
# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [22]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit
    data = data[:nbatch * bsz]
    # Evenly divide the data across the bsz batches
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [23]:
# batch_size = 20
# eval_batch_size = 10
# 
# 
# 
# # train_data = batchify(train_dataset, batch_size)
# # val_data = batchify(valid_dataset, eval_batch_size)
# # test_data = batchify(test_dataset, eval_batch_size)
# 
# test = list(train_dataset.values())
# print(test)

In [24]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [25]:
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.6023,  0.1092,  0.1417,  ..., -0.4177,  0.6059,  0.1764],
         [ 0.5119, -0.4770,  0.5508,  ..., -0.2814,  0.3793,  0.1156],
         [ 0.0995,  0.0867,  0.0869,  ...,  0.4789, -0.3236,  0.3122],
         ...,
         [ 0.8081, -0.7380,  0.2001,  ...,  0.7405, -0.7998,  0.6449],
         [ 0.3305, -0.1958,  0.3148,  ..., -0.0525,  0.5358,  0.1987],
         [ 0.5655, -0.2176, -0.4720,  ..., -0.3554,  0.6141, -0.2476]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.2835e-01,  6.0906e-01,  9.9998e-01, -9.9831e-01,  9.8739e-01,
          9.2930e-01,  9.9664e-01, -9.8909e-01, -9.9124e-01, -7.8317e-01,
          9.9476e-01,  9.9961e-01, -9.9850e-01, -9.9997e-01,  8.0649e-01,
         -9.9098e-01,  9.9545e-01, -6.9968e-01, -9.9999e-01, -2.6556e-01,
         -4.1867e-01, -9.9998e-01,  3.1562e-01,  9.7638e-01,  9.9257e-01,
          1.4021e-01,  9.9543e-01,  9.9999e-01,  9.4214e-01, -2.024

In [44]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
print(device)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


cuda
[{'entity': 'B-PER', 'score': np.float32(0.9990139), 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': np.float32(0.999645), 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [45]:


# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
count = 0
total = len(test_dataset)
for data in test_dataset:
    seq = " ".join([token for token in data['tokens']])
    ner_results = nlp(seq)
    j = 0
    print(seq)
    print(ner_results[0]['word'])
    for i in range(len(ner_results)):
        print("bert ner:",ner_results[i]['word'])
        print("true:", data['tokens'][j])
        while j < len(ner_results):
            if ner_results[i]['word'] == data['tokens'][j]:
                if ner_results[i]['entity'] == data['ner_tags'][j]:
                    print("bert ner:",ner_results[i]['word'])
                    count += 1
            j += 1
    break
        # print(ner_results[i]['entity'])
        # print(data)
        


    

Device set to use cuda:0


SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
J
bert ner: J
true: SOCCER
bert ner: ##AP
true: CHINA
bert ner: L
true: CHINA
bert ner: ##UC
true: CHINA
bert ner: CH
true: CHINA
bert ner: ##IN
true: CHINA
bert ner: ##A
true: CHINA


In [30]:
print(count/total)
print(count)

0.0
0


In [41]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity': 'B-PER', 'score': np.float32(0.9990139), 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': np.float32(0.999645), 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]
